In [3]:
# ===============================================================================================
# @des: Ensemble Kalmer Filter: We will later integrate this EnKF with several ICE sheet models.
# @date: 12-09-2024
# @author: Brian KYANJO
# ===============================================================================================

In [9]:
# import libraries --------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

## Steps to follow
- Start with replicating the Julia Code in Python
- This will help in comparing results later 
- After when python agrees with Julia
- Form General Classes for the code to make it more genral using OBP
- Profile the code to see how long the filter takes for very big vlaues of N
- Rewrite the Filter to C and call it in python 
- if Still slow, add parallel capabilities using MPI or OpenMP and still wrap it in python.
- Once this works, now proceed to trying to couple the filter with another problem
- If sucess then, now try with an actual ICE-sheet model
- Continue to remodel and develop the wrapper ....
- Finnally convert the wrapper into a python script

In [2]:
# Parameters definition

def params_define():
    # Prescribed initial values of model parameters in a dictionary
    params = {}
    params["A"]         = 4e-26         
    params["year"]      = 3600 * 24 * 365
    params["n"]         = 3
    params["C"]         = 3e6
    params["rho_i"]     = 900
    params["rho_w"]     = 1000
    params["g"]         = 9.8
    params["B"]         = params["A"] ** (-1 / params["n"])
    params["m"]         = 1 / params["n"]
    params["accum"]     = 0.65 / params["year"]
    params["facemelt"]  = 5 / params["year"]

    # Scaling parameters
    params["hscale"]    = 1000
    params["ascale"]    = 1.0 / params["year"]
    params["uscale"]    = (params["rho_i"] * params["g"] * params["hscale"] * params["ascale"] / params["C"]) ** (1 / (params["m"] + 1))
    params["xscale"]    = params["uscale"] * params["hscale"] / params["ascale"]
    params["tscale"]    = params["xscale"] / params["uscale"]
    params["eps"]       = params["B"] * ((params["uscale"] / params["xscale"]) ** (1 / params["n"])) / (2 * params["rho_i"] * params["g"] * params["hscale"])
    params["lambda"]    = 1 - (params["rho_i"] / params["rho_w"])

    # Grid parameters
    params["NT"]        = 1
    params["TF"]        = params["year"]
    params["dt"]        = params["TF"] / params["NT"]
    params["transient"] = 0
    params["tcurrent"]  = 1

    params["N1"]        = 40
    params["N2"]        = 10
    params["sigGZ"]     = 0.97
    params["NX"]        = params["N1"] + params["N2"]

    # Bed params
    # params["b0"] = -400
    params["xsill"]      = 50e3
    params["sillamp"]    = 500
    params["sillsmooth"] = 1e-5
    # params["bxr"] = 1e-3
    # params["bxp"] = -1e-3

    # EnKF params
    params["inflation"] = 1.0
    params["assim"]     = False

    # Generating sigma values
    sigma1    = np.linspace(params["sigGZ"] / (params["N1"] + 0.5), params["sigGZ"], int(params["N1"]))
    sigma2    = np.linspace(params["sigGZ"], 1, int(params["N2"] + 1))
    sigma     = np.concatenate((sigma1, sigma2[1:params["N2"] + 1]))

    # Create the grid dictionary
    grid                = {"sigma": sigma}
    grid["sigma_elem"]  = np.concatenate(([0], (sigma[:-1] + sigma[1:]) / 2))
    grid["dsigma"]      = np.diff(grid["sigma"])

    return params, grid

In [ ]:
# bed topography function --------------------------------------------------------------
def bed(x,params):
    b = params['sillamp'] * (-2 * np.arccos((1 - params['sillsmooth']) * np.sin(np.pi * x / (2 * params['xsill'])))/np.pi - 1)
    return b

In [ ]:
# Implicit flowline model function --------------------------------------------------------------
def flowline(F, varin, varin_old, params, grid, bedfun):
    # Unpack grid
    NX          = params["NX"]
    N1          = params["N1"]
    dt          = params["dt"] / params["tscale"]
    ds          = grid["dsigma"]
    sigma       = grid["sigma"]
    sigma_elem  = grid["sigma_elem"]

    # Unpack parameters
    tcurrent    = params["tcurrent"]
    xscale      = params["xscale"]
    hscale      = params["hscale"]
    lambd       = params["lambda"]
    m           = params["m"]
    n           = params["n"]
    a           = params["accum"] / params["ascale"]
    mdot        = params["facemelt"][tcurrent] / params["uscale"]
    eps         = params["eps"]
    transient   = params["transient"]

    # Unpack variables
    h   = varin[:NX]
    u   = varin[NX:2*NX]
    xg  = varin[2*NX]

    h_old   = varin_old[:NX]
    xg_old  = varin_old[2*NX]

    # Calculate bed
    hf  = -bedfun(xg * xscale, params) / (hscale * (1 - lambd))
    hfm = -bedfun(xg * sigma_elem[-1] * xscale, params) / (hscale * (1 - lambd))
    b   = -bedfun(xg * sigma * xscale, params) / hscale

    # Calculate thickness functions
    F[0] = transient * (h[0] - h_old[0]) / dt + (2 * h[0] * u[0]) / (ds[0] * xg) - a
    F[1] = (transient * (h[1] - h_old[1]) / dt
            - transient * sigma_elem[1] * (xg - xg_old) * (h[2] - h[0]) / (2 * dt * ds[1] * xg)
            + (h[1] * (u[1] + u[0])) / (2 * xg * ds[1]) - a)
    
    F[2:NX-1] = (transient * (h[2:NX-1] - h_old[2:NX-1]) / dt
                 - transient * sigma_elem[2:NX-1] * (xg - xg_old) * (h[3:NX] - h[1:NX-2]) / (2 * dt * ds[2:NX-1] * xg)
                 + (h[2:NX-1] * (u[2:NX-1] + u[1:NX-2]) - h[1:NX-2] * (u[1:NX-2] + u[0:NX-3])) / (2 * xg * ds[2:NX-1])
                 - a)

    F[N1-1] = (1 + 0.5 * (1 + (ds[N1-1] / ds[N1-2]))) * h[N1-1] - 0.5 * (1 + (ds[N1-1] / ds[N1-2])) * h[N1-2] - h[N1]
    F[NX-1] = (transient * (h[NX-1] - h_old[NX-1]) / dt
               - transient * sigma[NX-1] * (xg - xg_old) * (h[NX-1] - h[NX-2]) / (dt * ds[NX-2] * xg)
               + (h[NX-1] * (u[NX-1] + mdot * hf / h[NX-1] + u[NX-2]) - h[NX-2] * (u[NX-2] + u[NX-3])) / (2 * xg * ds[NX-2])
               - a)

    # Calculate velocity functions
    F[NX] = ((4 * eps / (xg * ds[0])**((1 / n) + 1)) * (h[1] * (u[1] - u[0]) * abs(u[1] - u[0])**((1 / n) - 1)
               - h[0] * (2 * u[0]) * abs(2 * u[0])**((1 / n) - 1))
             - u[0] * abs(u[0])**(m - 1)
             - 0.5 * (h[0] + h[1]) * (h[1] - b[1] - h[0] + b[0]) / (xg * ds[0]))

    F[NX+1:2*NX-1] = ((4 * eps / (xg * ds[1:NX-1])**((1 / n) + 1)) *
                      (h[2:NX] * (u[2:NX] - u[1:NX-1]) * abs(u[2:NX] - u[1:NX-1])**((1 / n) - 1)
                      - h[1:NX-1] * (u[1:NX-1] - u[0:NX-2]) * abs(u[1:NX-1] - u[0:NX-2])**((1 / n) - 1))
                      - u[1:NX-1] * abs(u[1:NX-1])**(m - 1)
                      - 0.5 * (h[1:NX-1] + h[2:NX]) * (h[2:NX] - b[2:NX] - h[1:NX-1] + b[1:NX-1]) / (xg * ds[1:NX-1]))

    F[NX+N1-1]  = (u[N1] - u[N1-1]) / ds[N1-1] - (u[N1-1] - u[N1-2]) / ds[N1-2]
    F[2*NX-1]   = (1 / (xg * ds[NX-2])**(1 / n)) * (abs(u[NX-1] - u[NX-2])**((1 / n) - 1)) * (u[NX-1] - u[NX-2]) - lambd * hf / (8 * eps)

    # Calculate grounding line functions
    F[2*NX] = 3 * h[NX-1] - h[NX-2] - 2 * hf


# Jacobian calculation using automatic differentiation
def Jac_calc(huxg_old, params, grid, bedfun, flowlinefun):
    def f(varin):
        F = np.zeros(2 * params["NX"] + 1)
        flowlinefun(F, varin, huxg_old, params, grid, bedfun)
        return F

    def Jf(J, varin):
        J[:, :] = optimize.approx_fprime(varin, f, epsilon=np.sqrt(np.finfo(float).eps))

    return Jf


In [ ]:
def flowline_run(varin, params, grid, bedfun, flowlinefun):
    nt = params["NT"]
    huxg_old = varin
    huxg_all = np.zeros((huxg_old.shape[0], nt))
    # huxg_all[:, 0] = huxg_old

    for i in range(nt):
        if not params["assim"]:
            params["tcurrent"] = i
        Jf = Jac_calc(huxg_old, params, grid, bedfun, flowlinefun)
        solve_result = optimize.root(flowlinefun, huxg_old, args=(huxg_old, params, grid, bedfun), jac=Jf, method='hybr')
        huxg_old = solve_result.x
        huxg_all[:, i] = huxg_old

        if not solve_result.success:
            err = "Solver didn't converge at time step " + str(i)
            print(err)
        if not params["assim"]:
            print("Step " + str(i) + "\n")

    return huxg_all

In [3]:
# Observation operator ----------------------
def Obs(huxg_virtual_obs,m_obs):
    n = huxg_virtual_obs.shape[0]
    m = m_obs
    H = np.zeros((m*2+1,n))
    di = int((n-2)/(2*m))  #distance between measurements
    for i in range(1,m+1):
        H[i-1,i*di] = 1
        H[m+i-1,int(((n-2)/2)+(i*di))] = 1
    H[m*2+1,n-1] = 1
    z = H.dot(huxg_virtual_obs)
    return z

In [ ]:
# Jacobian of the observation operator
def JObs(n_model, m_obs):
    n = n_model
    m = m_obs
    H = np.zeros((m*2+1, n))  # Create an array of zeros
    di = int((n - 2) / (2 * m))  # Calculate the distance between measurements

    for i in range(1, m+1):
        H[i-1, i*di] = 1  # Assign 1 to appropriate indices
        H[m + i - 1, int((n - 2) / 2 + i * di)] = 1
    
    H[m*2, n-1] = 1  # Set the last element in the matrix
    return H


In [ ]:
# EnKF function ------------------------------
def EnKF(huxg_ens,huxg_obs,ObsFun,JObsFun,Cov_obs,Cov_model,params,grid):
    n = huxg_ens.shape[0]  # State size
    N = huxg_ens.shape[1]  # Ensemble size
    m = huxg_obs.shape[0]  # Measurement size

    huxg_ens_mean = np.mean(huxg_ens, axis=1)   # Mean of model forecast ensemble
    Jobs = JObsFun(n, params["m_obs"])          # Jacobian of observation operator

    KalGain = Cov_model @ Jobs.T @ np.linalg.inv(Jobs @ Cov_model @ Jobs.T + Cov_obs)  # Compute Kalman gain

    obs_virtual = np.zeros((m, N))
    analysis_ens = np.zeros((n, N))
    # mismatch_ens = np.zeros((n-1, N))
    for i in range(N):
        obs_virtual[:, i] = huxg_obs + np.random.multivariate_normal(np.zeros(m), Cov_obs)  # Generate virtual observations
        analysis_ens[:, i] = huxg_ens[:, i] + KalGain @ (obs_virtual[:, i] - ObsFun(huxg_ens[:, i], params["m_obs"]))  # Generate analysis ensemble
        # mismatch_ens[:, i] = obs_virtual[:, i] - ObsFun(huxg_ens[:, i], params["m_obs"])  # Calculate mismatch

    analysis_ens_mean = np.mean(analysis_ens, axis=1)  # Mean of analysis ensemble

    analysis_cov = (1 / (N - 1)) * (analysis_ens - np.repeat(analysis_ens_mean, N, axis=1)) @ (analysis_ens - np.repeat(analysis_ens_mean, N, axis=1)).T  # Analysis error covariance

    statevec_sig = np.concatenate((grid["sigma_elem"], grid["sigma"], np.array([1, 1])))
    taper = np.ones((statevec_sig.shape[0], statevec_sig.shape[0]))

    taper[-1, -3] = 2  
    taper[-3, -1] = 2  
    taper[-1, -1] = 10  
    taper[-2, -1] = 10  
    taper[-1, -2] = 10  

    analysis_cov = analysis_cov * taper
    return analysis_ens, analysis_cov
    

In [4]:
# call the parameters function
params, grid = params_define()

In [8]:
# wrong simulation -------------------------------------------------
fm_wrong = np.linspace(5, 45, params["NT"] + 1) / params["year"]
params["facemelt"] = np.linspace(5, 45, params["NT"] + 1) / params["year"]

ts = np.linspace(0, params["TF"] / params["year"], params["NT"] + 1)
# xg_truth = np.concatenate((huxg_out0[2 * params["NX"]], huxg_out1[2 * params["NX"], :])) * params["xscale"]

array([1.58548960e-07, 1.42694064e-06])

In [1]:
# # The Ensemble Kalman Filter (EnKF) class ---------------------------------------
# class EnKF:
#     '''
#     Parameters:
#     -----------
#     n = number of state variables
#     m = number of observations
#     R = observation error covariance matrix
#     dt = time step
#     Q = process noise covariance matrix
#     P = state covariance matrix
#     x = state vector
#     X = state ensemble
#     Y = observation ensemble
#     K = Kalman gain
#     y = observation vector
#     yo = observation mean
#     xo = state mean
#     '''
#     def __init__(self, model, n, m, R, dt):
#         self.model = model
#         self.n = n
#         self.m = m
#         self.R = R
#         self.dt = dt
#         self.Q = np.eye(n) * 1e-5
#         self.P = np.eye(n)
#         self.x = np.random.randn(n)
#         self.X = np.random.randn(n, n)
#         self.Y = np.random.randn(m, n)
#         self.K = np.zeros((n, m))
#         self.y = np.zeros(m)
#         self.yo = np.zeros(m)
#         self.xo = np.zeros(n)

#     def forecast(self):
#         self.X = self.model(self.X, self.dt) + np.random.multivariate_normal(np.zeros(self.n), self.Q, self.n).T

#     def analysis(self, y):
#         self.yo = y
#         self.xo = np.mean(self.X, axis=1)
#         self.Y = self.model(self.X, self.dt)
#         self.y = np.mean(self.Y, axis=1)
#         Pyy = np.cov(self.Y) + self.R
#         Pxy = np.cov(self.X, self.Y)
#         self.K = np.dot(Pxy, np.linalg.inv(Pyy))
#         self.X = self.X + np.dot(self.K, (y - self.y))
#         self.P = np.cov(self.X)

